In [1]:
# from __future__ import division
# from __future__ import print_function
import caffe
import numpy as np
import os
import sys
from datetime import datetime
from PIL import Image
from IPython.display import Image
from IPython.display import clear_output
import time


import os.path as osp
from easydict import EasyDict as edict
import yaml

In [2]:
def log_progress(sequence, title='', every=1, size=None, end_msg='', removeAtTheEnd=False):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    if isinstance(sequence, dict):
                        val = ''
                    else:
                        val=sequence[index-1]
                            
                    label.value = u'{title} : {value} ({index} / {size})'.format(
                        title=title,
                        value=val,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = end_msg
        if removeAtTheEnd:
            box.close()
    

In [3]:
def yaml2xml(yamlfile, xmlfile):
    with open(yamlfile) as fin:
        c = fin.read()
        # some operator on raw conent of c may be needed
        c = "%YAML 1.1"+os.linesep+"---" + c[len("%YAML:1.0"):] if c.startswith("%YAML:1.0") else c
        result = edict(yaml.load(c))
    
    
    fd=open(xmlfile,'w')
    print >> fd, "<annotation>"
    print >> fd, "	<folder>Depth</folder>"
    print >> fd, "	<filename>"+result.annotation.filename+"</filename>"
    print >> fd, "	<source>"
    print >> fd, "		<database>Freiburg RGB-D People Dataset</database>"
    print >> fd, "	</source>"
    print >> fd, "	<owner>Andreas Eitel,  Oier Mees</owner>"
    print >> fd, "	<size>"
    print >> fd, "		<width>1920</width>"
    print >> fd, "		<height>1080</height>"
    print >> fd, "		<depth>1</depth>"
    print >> fd, "	</size>"
    print >> fd, "	<segmented>0</segmented>"

    if 'object' in result.annotation:
        for obj in result.annotation.object:
            print >> fd, "	<object>"
            print >> fd, "		<name>"+obj.name+"</name>"
            print >> fd, "		<pose>unknown</pose>"
            print >> fd, "		<truncated>-1</truncated>"
            print >> fd, "		<difficult>0</difficult>"
            print >> fd, "		<bndbox>"
            print >> fd, "			<xmin>"+obj.bndbox.xmin+"</xmin>"
            print >> fd, "			<ymin>"+obj.bndbox.ymin+"</ymin>"
            print >> fd, "			<xmax>"+obj.bndbox.xmax+"</xmax>"
            print >> fd, "			<ymax>"+obj.bndbox.ymax+"</ymax>"
            print >> fd, "		</bndbox>"
            print >> fd, "	</object>"

    print >> fd, "</annotation>"
    fd.close()

In [4]:
xml_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/Annotations_pv/'
splits_basic_path = '/home/jogue/workspace/datasets/InOutDoorPeople/ImageSets/'
yaml_basic_path='/home/jogue/workspace/datasets/InOutDoorPeople/Annotations/'

splits=['seq0.txt', 'seq1.txt',  'seq2.txt',  'seq3.txt']



In [5]:
!cat /c16/THESE.JORIS/datasets/InOutDoorPeople/Annotations/seq_1454064092.6794846100.yml

%YAML:1.0
annotation:
  filename: seq_1454064092.6794846100.jpg
  folder: VOC2007
  owner: Andreas Eitel,  Oier Mees
  segmented: '0'
  size: {depth: '1', height: '1080', width: '1920'}
  source: '{database: Freiburg RGB-D People Dataset}'


In [5]:
for split in log_progress(splits, title='Splits', end_msg='Finished !',removeAtTheEnd=False):
    list_images = open(splits_basic_path+split, 'r').read().splitlines()
    for img_name in log_progress(list_images, title='img', end_msg='Finished !',removeAtTheEnd=True):
        xmlfile = xml_basic_path+img_name+'.xml'
        yamlfile=yaml_basic_path+img_name+'.yml'
        yaml2xml(yamlfile, xmlfile)